In [ ]:
# Automatically reload custom code modules when there are changes:
%load_ext autoreload
%autoreload 2


In [ ]:
# Adjust relative path so that the notebook can find the code modules:
import sys
sys.path.append('code/')


In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
# Import code modules:
from structures import *


In [ ]:
# Hide warnings about safe distance violation (still in development):
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
# Define a ring road environment:
env = RingRoad(
    num_vehicles = 22,  # The vechicles at index 0 is an A.V.
    ring_length = 230.0,  # The road is a cicle.
    starting_noise = 4.0,  # Uniformly add noise to starting positions.
    temporal_res = 0.3,  # Set the size of simulation steps (seconds).
    av_activate = 30,  # Set when the PID controller is activated.
    seed = 286,  # Set a random seed.
)

# Plot initial conditions:
fig,ax = env.visualize(step=0, draw_cars_to_scale=True, draw_safety_buffer=True)


In [ ]:
# Run the simulation for set number of time steps:
total_time = 50  # In seconds.
total_steps = int(np.ceil(total_time/env.dt))
env.run(steps=total_steps)


In [ ]:
step = env.step  # Final step of simulation.
fig, axs = env.plot_dashboad(
    step=int(np.ceil(step/env.dt)), total_steps=total_steps,
    draw_cars_to_scale=True, draw_safety_buffer=True, label_step=True, label_cars=True,
)


In [ ]:
steps_before = range(0,env.av_activate)
steps_after = range(env.av_activate, env.step)
speeds_before = env.get_vehicle_vel_table(steps_before)
speeds_after = env.get_vehicle_vel_table(steps_after)
print("Mean standard deviation before: {}".format(speeds_before.std(axis=0).mean()))
print("Mean standard deviation after: {}".format(speeds_after.std(axis=0).mean()))
df_before = pd.DataFrame(speeds_before.std(axis=0).to_frame(name='before AV control'))
df_after = pd.DataFrame(speeds_after.std(axis=0).to_frame(name='after AV control'))
pd.concat([df_before,df_after],axis=1)


In [ ]:
# n = 10  # Draw the ring road at every n-th step.
# for step in np.arange(0,env.step,int(n)):
#     env.visualize(step=step, draw_cars_to_scale=False, label_cars=True)
